In [3]:
from heapq import *
from collections import defaultdict

def main():
    # n: 200， m: 10
    n,m = map(int,input().split())
    # i番目(1-indexed)の山の下からj(1-indexed)番目にある数字，(一番下に300があるとする)
    B = [[]] + [[300]+list(map(int,input().split())) for _ in range(m)]
    # num_ind: {key:数字, value:[その数字が何番目の山にあるか, その山の下から何番目にあるか]}
    num_ind = defaultdict(list)
    for i in range(1,m+1):
        for j in range(1,n//m+1):
            num_ind[B[i][j]] = [i,j]
    # moved: i番目の山が移動されてないなら-1，移動されているなら移動された側の数字を記録
    moved = [-1]*(m+1)
    # 取り出したい数字
    hope_num = 1

    while hope_num <= 200:
        # print(moved)
        # 各山の一番上にhope_numがあるかどうか，あったら取り出してhope_numを更新して次のループへ
        flag = False
        for i in range(1,m+1):
            if B[i][-1] == hope_num:
                B[i].pop()
                print(hope_num, 0)
                if len(B[i]) == 1:
                    # print(moved)
                    # exit()
                    # hope_num+1の上の塊をi番目の山に移動
                    if hope_num != 200:
                        # 移動させる山
                        move_mt = num_ind[hope_num+1][0]
                        move_ind = len(B[move_mt])-1
                        if moved[move_mt] != -1:
                            move_ind = num_ind[moved[move_mt]][1]
                        print(moved[move_mt], i)
                        B[i] += B[move_mt][move_ind:]
                        B[move_mt] = B[move_mt][:move_ind]
                        for j,b in enumerate(B[i][1:], start=1):
                            num_ind[b] = [i, j]
                        moved[i] = moved[move_mt]
                        moved[move_mt] = -1
                        # print(moved)
                        # exit()
                # print(moved)
                # if moved[i] == hope_num:
                #     moved[i] = -1
                flag = True
                hope_num += 1
                break
        if flag:
            continue
        # 全ての山の一番上の数字うち一番大きい数
        all_1 = 0
        # 移動されていない山の一番上の数字うち一番大きい数
        not_moved_1 = 0
        # 移動されていない山の一番上の数字うち二番目に大きい数
        not_moved_2 = 0
        for i in range(1,m+1):
            if moved[i] == -1:
                if not_moved_1 < B[i][-1]:
                    not_moved_2 = not_moved_1
                    not_moved_1 = B[i][-1]
                elif not_moved_2 < B[i][-1]:
                    not_moved_2 = B[i][-1]
            if all_1 < B[i][-1]:
                all_1 = B[i][-1]
        # 選んだ数字をその数字より大きいかつ最も近い数字の山に移動
        selected_num = 0
        # 全ての山の一番大きい数と，移動されていない山の一番大きい数が違うなら，移動されていない山の一番大きい数を選ぶ
        if all_1 != not_moved_1:
            selected_num = not_moved_1
        # 全ての山の一番大きい数と，移動されていない山の一番大きい数が同じなら，移動されていない山の二番目に大きい数を選ぶ
        else:
            if not_moved_2 != 0:
                selected_num = not_moved_2
            else:
                # マージする
                # hope_numの山
                hope_mt = num_ind[hope_num][0]
                # hope_numの山の上のかたまりの土台の数字
                num1 = moved[hope_mt]
                num1_mt, num1_ind = hope_mt,len(B[hope_mt])-1
                if num1 != -1:
                    num1_mt, num1_ind = num_ind[num1]
                # hope_numの山以外で最もかたまりの数が少ない山
                num2_mt, num2_ind = 0,0
                # かたまりの数
                num2_cnt = 500
                for i in range(1,m+1):
                    if i == hope_mt:
                        continue
                    if moved[i] == -1:
                        continue
                    # print(moved)
                    # print(i,len(B[i]))
                    tmp_cnt = len(B[i]) - num_ind[moved[i]][1]
                    if tmp_cnt < num2_cnt:
                        num2_cnt = tmp_cnt
                        num2_mt, num2_ind = num_ind[moved[i]]
                # num1_mtとnum2_mt以外でマージのためにとりあえず置いとく山
                tmp_mt = set(range(1,5))
                tmp_mt.discard(num1_mt)
                tmp_mt.discard(num2_mt)
                tmp1_mt = tmp_mt.pop()
                tmp2_mt = tmp_mt.pop()
                # num1_mtのnum1_ind以降を上から一つずつtmp1_mtに移動
                # 実装上はヒープで[-1*数字]を管理して大きい方から取り出す
                heap = []
                for _ in range(num1_ind, len(B[num1_mt])):
                    # 取り出す数字
                    move_num = B[num1_mt].pop()
                    print(move_num, tmp1_mt)
                    heappush(heap, -1*move_num)
                # num2_mtのnum2_ind以降を上から一つずつtmp2_mtに移動
                for _ in range(num2_ind, len(B[num2_mt])):
                    # 取り出す数字
                    move_num = B[num2_mt].pop()
                    print(move_num, tmp2_mt)
                    heappush(heap, -1*move_num)
                # マージした結果をnum2_mtに移動
                while heap:
                    move_num = -1*heappop(heap)
                    print(move_num, num2_mt)
                    num_ind[move_num] = [num2_mt, len(B[num2_mt])]
                    B[num2_mt].append(move_num)
                # 移動された山の記録
                moved[num2_mt] = max(moved[num1_mt], moved[num2_mt])
                moved[num1_mt] = -1
                continue
        # selected_numより大きいかつ最も近い数字
        selected_num2 = 500
        for i in range(1,m+1):
            if selected_num < B[i][-1] and B[i][-1] < selected_num2:
                selected_num2 = B[i][-1]
        # 空いている山があるなら，かつ，
        # moved[num_ind[hope_num][0]]が-1でないなら，moved[num_ind[hope_num][0]]より上の数字をまとめて空いている山に移動させて次のループへ
        # moved[num_ind[hope_num][0]]が-1なら，処理を続ける
        if selected_num2 == 300:
            if moved[num_ind[hope_num][0]] != -1:
                # 移動させる数字
                move_num = moved[num_ind[hope_num][0]]
                # 移動させる山とindex
                move_mt, move_ind = num_ind[move_num]
                # 空いている山の番号
                num2_mt = 0
                for i in range(1,m+1):
                    if B[i][-1] == 300:
                        num2_mt = i
                        break
                print(move_num, num2_mt)
                moved[num2_mt] = move_num
                moved[move_mt] = -1
                B[num2_mt] += B[move_mt][move_ind:]
                B[move_mt] = B[move_mt][:move_ind]
                for i,b in enumerate(B[num2_mt][1:], start=1):
                    num_ind[b] = [num2_mt, i]
                continue
        # selected_numの山とindex
        num1_mt, num1_ind = num_ind[selected_num]
        # selected_num2の山とindex
        num2_mt, num2_ind = 0,0
        if selected_num2 != 300:
            num2_mt, num2_ind = num_ind[selected_num2]
        else:
            num2_ind = 0
            for i in range(1,m+1):
                if B[i][-1] == 300:
                    num2_mt = i
                    break
        # selected_numをselected_num2の山に移動
        B[num1_mt].pop()
        B[num2_mt].append(selected_num)
        # 移動された山の記録
        if moved[num2_mt] == -1:
            moved[num2_mt] = selected_num2
        if len(B[num2_mt]) == 2:
            moved[num2_mt] = -1
        num_ind[selected_num] = [num2_mt, num2_ind+1]
        print(selected_num, num2_mt)

if __name__ == "__main__":
    main()

defaultdict(<class 'list'>, {1: [[1, 1]], 2: [[1, 2]], 3: [[1, 3]], 5: [[2, 1]], 4: [[2, 2]], 6: [[2, 3]], 9: [[3, 1]], 8: [[3, 2]], 7: [[3, 3]]})


In [2]:
m = 3
A = [[]] + [[300]+list(map(int,input().split())) for _ in range(m)]
print("start")
for a in A:
    print(*a)

start

300 1 2 3
300 4 3 2
300 6 4 3


In [ ]:
55 3

4 2
168 5
70 5
2 2
11 8
129 1
70 6
168 6
171 6
52 3
57 3
106 3
142 3
171 5
168 5
142 5
106 5
70 5
57 5
52 5
94 1
137 4
